# Problem01. Searching Entrez

## 1. Summary

### 1-1. Searcing Entrez

Entrez는 대표적인 생물정보포털 사이트인 NCBI의 검색엔진이다. 이를 이용하여 비만과 식이요법과 관련된 논문을 검색하고 그 목록을 구해본다.


#### purpose

- NCBI의 데이터베이스 구조를 이해하고 검색엔진의 쿼리 방법을 이해한다.
- UID의 의미와 그 목록의 다운로드를 이해한다.

#### outputs

1. 웹사이트 화면 : NCBI 홈페이지 접속 화면
1. 쿼리어 : 조회한 쿼리 문
1. 목록 화면 : 조회된 검색결과 목록
1. UID 목록화면 : 전체검과 중 UID 목록 화면
1. UID 목록파일 : 전체검과 중 UID를 저장한 파일

#### deadline
1월 25일 금요일

----

## 2. Processing

### 2-1. Setting

#### 2-1-1. Biopython
Entrez을 사용하기 위해서는 Biopython을 설치해야 한다. 우선 pip를 업데이트한 후 biopython을 설치한다.


In [3]:
!python -m pip install -U pip
!pip install -U biopython

Requirement already up-to-date: pip in c:\users\eunchul jang\appdata\local\programs\python\python36\lib\site-packages (19.0.1)
Requirement already up-to-date: biopython in c:\users\eunchul jang\appdata\local\programs\python\python36\lib\site-packages (1.73)


#### 2-1-2. Library, Mail
Biopython에서 Entrez을 불러온다. 그리고 Entrez 사용하기 위해서는 사용자의 메일을 입력해야한다. 꼭 본인 메일일 필요는 없으며 여기서는 jang@gmail.com을 사용하였다.

In [7]:
from Bio import Entrez, SeqIO
Entrez.email = "jang@gmail.com"

### 2-2. Connect Entrez

#### 2-2-1. Einfo
Entrz에 접속한 후 제공되는 DB의 종류를 알아본다. 
- **einfo** : 데이터베이스의 종류를 사전형태로 반환
- DbList라는 키값으로 저장, 총 49가지의 DB 확인

In [18]:
search_db_dic = Entrez.read(Entrez.einfo()) 

In [54]:
search_db_dic['DbList']

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'nucgss', 'nucest', 'structure', 'sparcle', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'clone', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosystems', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'unigene', 'gencoll', 'gtr']

In [33]:
import pandas as pd

In [43]:
search_db_list = pd.DataFrame(search_db_dic)
search_db_list.shape

(49, 1)

홈페이지에 있는 DB 목록과 일치한다.

| DB List | Result |
| --- | --- |
| ![01-001-dblist](./img/01-001-dblist.png) | ![01-002-dblist_result](./img/01-002-dblist_result.png) |

### 2-3. Search

#### 2-3-1. result

esarch 기능을 사용하여 검색어를 입력한 목록을 불러올 수 있다.
- 검색어: 비만, 식이요법 / obesity, diet
- 검색데이터베이스: pubmed
- 검색 결과 총 65048개, 실제 검색결과와 일치

In [57]:
od_result = Entrez.read(Entrez.esearch(db="pubmed", term="obesity AND diet")

In [62]:
for each_key in od_result.keys():
    print(each_key,": ", od_result[each_key])

Count :  65048
RetMax :  20
RetStart :  0
IdList :  ['30673851', '30673669', '30673504', '30673296', '30672742', '30672719', '30672576', '30672127', '30671370', '30671117', '30670942', '30670847', '30670846', '30670780', '30670753', '30670473', '30670159', '30669573', '30669549', '30669348']
TranslationSet :  [DictElement({'From': 'obesity', 'To': '"obesity"[MeSH Terms] OR "obesity"[All Fields]'}, attributes={}), DictElement({'From': 'diet', 'To': '"diet"[MeSH Terms] OR "diet"[All Fields]'}, attributes={})]
TranslationStack :  [DictElement({'Term': '"obesity"[MeSH Terms]', 'Field': 'MeSH Terms', 'Count': '192944', 'Explode': 'Y'}, attributes={}), DictElement({'Term': '"obesity"[All Fields]', 'Field': 'All Fields', 'Count': '297717', 'Explode': 'N'}, attributes={}), 'OR', 'GROUP', DictElement({'Term': '"diet"[MeSH Terms]', 'Field': 'MeSH Terms', 'Count': '258221', 'Explode': 'Y'}, attributes={}), DictElement({'Term': '"diet"[All Fields]', 'Field': 'All Fields', 'Count': '439391', 'Explo

홈페이지 검색 결과 : https://www.ncbi.nlm.nih.gov/pubmed/?term=obesity+AND+diet

![01-003-ob_search](./img/01-003-ob_search.png)

#### 2-3-2. UID List
조회한 결과중에서 ID 목록만 따로 추출하면 화면에 보이는 20개만 추출된다. 전체 검색결과 수를 입력해주어야 한다.

- 검색결과 개수: Count, ID 리스트: IdList
- retmax에서 retmax 기능 사용
- 총 65048개의 ID 목록을 생성

In [78]:
od_result_count = od_result["Count"]
od_result_count

'65048'

In [79]:
od_result_all = Entrez.read(
    Entrez.esearch(
        db="pubmed", 
        term="obesity AND diet", 
        retmax=od_result_count
        )
    )

In [86]:
od_result_ids_list = pd.DataFrame(od_result_all['IdList'], columns=['IdList'])
od_result_ids_list.shape

(65048, 1)

In [87]:
od_result_ids_list.head()

,IdList
0,30673851
1,30673669
2,30673504
3,30673296
4,30672742


### 2-4. Save Result

#### 2-4-1. Save
판다스의 엑셀 저장 기능을 사용하여 저장한다.

In [91]:
od_result_ids_list.to_csv("od_result_ids_out.csv", mode="w")

저장된 파일을 불러와서 확인해본다.

In [92]:
ids_out_read = pd.read_csv("od_result_ids_out.csv", index_col=0)
ids_out_read.head()

,IdList
0,30673851
1,30673669
2,30673504
3,30673296
4,30672742


![01-004-excel](./img/01-004-excel.png)

----